In [1]:
# default_exp trainer.tacotron2

In [1]:
# export
from random import choice, randint
from uberduck_ml_dev.models.tacotron2.tacotron2 import Tacotron2
import time
from typing import List

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

from uberduck_ml_dev.data_loader import TextMelDataset, TextMelCollate

from uberduck_ml_dev.utils.plot import save_figure_to_numpy
from uberduck_ml_dev.utils.utils import reduce_tensor
from uberduck_ml_dev.monitoring.statistics import get_alignment_metrics
import nemo
from typing import NamedTuple
from uberduck_ml_dev.losses.tacotron2 import Tacotron2Loss

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
# export
import os
from pathlib import Path
from pprint import pprint
from random import choice

import torch
from torch.cuda.amp import autocast, GradScaler
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from tensorboardX import SummaryWriter
import time
from torch.utils.data import DataLoader
from uberduck_ml_dev.models.common import MelSTFT
from uberduck_ml_dev.models.torchmoji import TorchMojiInterface
from uberduck_ml_dev.utils.plot import (
    plot_attention,
    plot_gate_outputs,
    plot_spectrogram,
)
from uberduck_ml_dev.text.util import text_to_sequence, random_utterance
from uberduck_ml_dev.trainer.base import TTSTrainer
from uberduck_ml_dev.data_loader import TextMelDataset, TextMelCollate
from uberduck_ml_dev.data.batch import Batch

class Tacotron2Trainer(TTSTrainer):

    REQUIRED_HPARAMS = [
        "audiopaths_and_text",
        "checkpoint_path",
        "epochs",
        "mel_fmax",
        "mel_fmin",
        "n_mel_channels",
        "text_cleaners",
        "pos_weight",
    ]

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.include_durations = self.hparams.include_durations
        self.sampling_rate = self.hparams.sampling_rate
        self.hop_length = self.hparams.hop_length

        if self.hparams.get("gst_type") == "torchmoji":
            assert self.hparams.get(
                "torchmoji_vocabulary_file"
            ), "torchmoji_vocabulary_file must be set"
            assert self.hparams.get(
                "torchmoji_model_file"
            ), "torchmoji_model_file must be set"
            assert self.hparams.get("gst_dim"), "gst_dim must be set"

            self.torchmoji = TorchMojiInterface(
                self.hparams.get("torchmoji_vocabulary_file"),
                self.hparams.get("torchmoji_model_file"),
            )
            self.compute_gst = lambda texts: self.torchmoji.encode_texts(texts)
        else:
            self.compute_gst = None

        if not self.sample_inference_speaker_ids:
            self.sample_inference_speaker_ids = list(range(self.n_speakers))

        # pass

    def log_training(
        self,
        model,
        X,
        y_pred,
        y,
        loss,
        mel_loss,
        gate_loss,
        mel_loss_batch,
        gate_loss_batch,
        grad_norm,
        step_duration_seconds,
    ):
        self.log("Loss/train", self.global_step, scalar=loss)
        self.log("MelLoss/train", self.global_step, scalar=mel_loss)
        self.log("GateLoss/train", self.global_step, scalar=gate_loss)
        self.log("GradNorm", self.global_step, scalar=grad_norm.item())
        self.log("LearningRate", self.global_step, scalar=self.learning_rate)
        self.log(
            "StepDurationSeconds", self.global_step, scalar=step_duration_seconds,
        )

        batch_levels = X.speaker_ids
        batch_levels_unique = torch.unique(batch_levels)
        for l in batch_levels_unique:
            mlb = mel_loss_batch[torch.where(batch_levels == l)[0]].mean()
            self.log(
                f"MelLoss/train/speaker{l.item()}", self.global_step, scalar=mlb,
            )
            if self.location_specific_attention:
                glb = gate_loss_batch[torch.where(batch_levels == l)[0]].mean()
                self.log(
                    f"GateLoss/train/speaker{l.item()}", self.global_step, scalar=glb,
                )
                self.log(
                    f"Loss/train/speaker{l.item()}", self.global_step, scalar=mlb + glb,
                )

        if self.global_step % self.steps_per_sample == 0:

            mel_out_postnet = y_pred.mel_outputs_postnet
            
            sample_idx = randint(0, mel_out_postnet.size(0) - 1)
            audio = self.sample(mel=mel_out_postnet[sample_idx])
            input_length = X.input_lengths[sample_idx].item()
            output_length = X.output_lengths[sample_idx].item()
            gate_pred = y_pred.gate_pred
            alignments = y_pred.alignments
            mel_target = y.mel_padded
            gate_target = y.gate_target
            if self.location_specific_attention:
                alignment_metrics = get_alignment_metrics(alignments)
                alignment_diagonalness = alignment_metrics["diagonalness"]
                alignment_max = alignment_metrics["max"]
                self.log(
                    "AlignmentDiagonalness/train",
                    self.global_step,
                    scalar=alignment_diagonalness,
                )
                self.log("AlignmentMax/train", self.global_step, scalar=alignment_max)
                self.log("AudioSample/train", self.global_step, audio=audio)
                self.log(
                    "Attention/train",
                    self.global_step,
                    image=save_figure_to_numpy(
                        plot_attention(
                            alignments[sample_idx].data.cpu().transpose(0, 1),
                            encoder_length=input_length,
                            decoder_length=output_length,
                        )
                    ),
                )
                self.log(
                    "Gate/train",
                    self.global_step,
                    image=save_figure_to_numpy(
                        plot_gate_outputs(
                            gate_targets=gate_target[sample_idx].data.cpu(),
                            gate_outputs=gate_pred[sample_idx].data.cpu(),
                        )
                    ),
                )
            alignment_max = alignment_metrics["max"]
            audio = self.sample(mel=mel_out_postnet[sample_idx])

            self.log(
                "MelPredicted/train",
                self.global_step,
                image=save_figure_to_numpy(
                    plot_spectrogram(mel_out_postnet[sample_idx].data.cpu())
                ),
            )
            self.log(
                "MelTarget/train",
                self.global_step,
                image=save_figure_to_numpy(
                    plot_spectrogram(mel_target[sample_idx].data.cpu())
                ),
            )



            for speaker_id in self.sample_inference_speaker_ids:
                if self.distributed_run:
                    self.sample_inference(
                        model.module, self.sample_inference_text, speaker_id,
                    )
                else:
                    self.sample_inference(
                        model, self.sample_inference_text, speaker_id,
                    )

    def sample_inference(self, model, transcription=None, speaker_id=None):
        if self.rank is not None and self.rank != 0:
            return
        # Generate an audio sample
        with torch.no_grad():
            if transcription is None:
                transcription = random_utterance()

            if self.compute_gst:
                gst_embedding = self.compute_gst([transcription])
                gst_embedding = torch.FloatTensor(gst_embedding).cuda()
            else:
                gst_embedding = None

            utterance = torch.LongTensor(
                text_to_sequence(
                    transcription,
                    self.text_cleaners,
                    p_arpabet=self.p_arpabet,
                    symbol_set=self.symbol_set,
                )
            )[None].cuda()

            input_lengths = torch.LongTensor([utterance.shape[1]]).cuda()
#             input_ = [
#                 utterance,
#                 input_lengths,
#                 torch.LongTensor([speaker_id]).cuda(),
#                 gst_embedding,
#             ]
            
            input_ = Batch(text = utterance,
                          input_lengths = input_lengths,
                          speaker_ids = torch.LongTensor([speaker_id]).cuda(),
                          gst = gst_embedding)
        
            model.eval()

            output = model.inference(input_)

            model.train()
            try:
                audio = self.sample(output.mel_outputs_postnet[0])
                self.log(f"SampleInference/{speaker_id}", self.global_step, audio=audio)
            except Exception as e:
                print(f"Exception raised while doing sample inference: {e}")
                print("Mel shape: ", output.mel_outputs_postnet[0].shape)
            self.log(
                f"Attention/{speaker_id}/sample_inference",
                self.global_step,
                image=save_figure_to_numpy(
                    plot_attention(output.alignments[0].data.cpu().transpose(0, 1))
                ),
            )
            self.log(
                f"MelPredicted/{speaker_id}/sample_inference",
                self.global_step,
                image=save_figure_to_numpy(plot_spectrogram(output.mel_outputs_postnet[0].data.cpu())),
            )
            self.log(
                f"Gate/{speaker_id}/sample_inference",
                self.global_step,
                image=save_figure_to_numpy(
                    plot_gate_outputs(gate_outputs=output.gate_pred[0].data.cpu())
                ),
            )

    def log_validation(
        self,
        X,
        y_pred,
        y,
        mean_loss,
        mean_mel_loss,
        mean_gate_loss,
        batch_mel_loss,
        batch_gate_loss,
        speakers_val,
    ):

        self.log("Loss/val", self.global_step, scalar=mean_loss)
        self.log("MelLoss/val", self.global_step, scalar=mean_mel_loss)
        self.log("GateLoss/val", self.global_step, scalar=mean_gate_loss)

        val_levels = speakers_val
        val_levels_unique = torch.unique(val_levels)
        for lev in val_levels_unique:
            mlv = (
                batch_mel_loss[torch.where(val_levels == lev)[0]]
                .mean()
                .cpu()
                .detach()
                .numpy()
            )
            self.log(
                f"MelLoss/val/speaker{lev.item()}", self.global_step, scalar=mlv,
            )
            if self.location_specific_attention:
                glv = (
                    batch_gate_loss[torch.where(val_levels == lev)[0]]
                    .mean()
                    .cpu()
                    .detach()
                    .numpy()
                )
                self.log(
                    f"GateLoss/val/speaker{lev.item()}", self.global_step, scalar=glv,
                )
                lv = mlv + glv
            # NOTE (Sam): add duration loss
            if self.non_attentive:
                lv = mlv
            self.log(
                f"Loss/val/speaker{lev.item()}", self.global_step, scalar=lv,
            )
        # Generate the sample from a random item from the last y_pred batch.
        mel_target = y.mel_padded
        gate_target = y.gate_target
        alignments = y_pred.alignments
        gate_pred = y_pred.gate_pred
        mel_out_postnet = y_pred.mel_outputs_postnet
        sample_idx = randint(0, mel_out_postnet.size(0) - 1)

        if self.location_specific_attention:
            alignment_metrics = get_alignment_metrics(alignments)
            alignment_diagonalness = alignment_metrics["diagonalness"]
            alignment_max = alignment_metrics["max"]
            self.log(
                "AlignmentDiagonalness/val",
                self.global_step,
                scalar=alignment_diagonalness,
            )
            self.log("AlignmentMax/val", self.global_step, scalar=alignment_max)
            self.log(
                "Gate/val",
                self.global_step,
                image=save_figure_to_numpy(
                    plot_gate_outputs(
                        gate_targets=gate_target[sample_idx].data.cpu(),
                        gate_outputs=gate_pred[sample_idx].data.cpu(),
                    )
                ),
            )
            input_length = X.input_lengths[sample_idx].item()
            output_length = X.output_lengths[sample_idx].item()
            self.log(
                "Attention/val",
                self.global_step,
                image=save_figure_to_numpy(
                    plot_attention(
                        alignments[sample_idx].data.cpu().transpose(0, 1),
                        encoder_length=input_length,
                        decoder_length=output_length,
                    )
                ),
            )

        audio = self.sample(mel=mel_out_postnet[sample_idx])

        self.log("AudioSample/val", self.global_step, audio=audio)
        self.log(
            "MelPredicted/val",
            self.global_step,
            image=save_figure_to_numpy(
                plot_spectrogram(mel_out_postnet[sample_idx].data.cpu())
            ),
        )
        self.log(
            "MelTarget/val",
            self.global_step,
            image=save_figure_to_numpy(
                plot_spectrogram(mel_target[sample_idx].data.cpu())
            ),
        )

    def initialize_loader(
        self,
        include_f0: bool = False,
        include_durations: bool = False,
        n_frames_per_step: int = 1,
    ):
        train_set = TextMelDataset(
            **self.training_dataset_args,
            debug=self.debug,
            debug_dataset_size=self.batch_size,
        )
        val_set = TextMelDataset(
            **self.val_dataset_args,
            debug=self.debug,
            debug_dataset_size=self.batch_size,
        )
        collate_fn = TextMelCollate(
            n_frames_per_step=n_frames_per_step,
            include_f0=include_f0,
            include_durations=include_durations,
        )
        sampler = None
        if self.distributed_run:
            self.init_distributed()
            sampler = DistributedSampler(train_set, rank=self.rank)
        train_loader = DataLoader(
            train_set,
            batch_size=self.batch_size,
            shuffle=(sampler is None),
            sampler=sampler,
            collate_fn=collate_fn,
        )
        return train_set, val_set, train_loader, sampler, collate_fn

    def train(self):
        train_start_time = time.perf_counter()
        print("start train", train_start_time)
        print("campaign with this game vn1")
        train_set, val_set, train_loader, sampler, collate_fn = self.initialize_loader(
            include_f0=self.include_f0, include_durations=self.include_durations
        )
        print("campaign with this game v0")
        criterion = Tacotron2Loss(
            pos_weight=self.pos_weight,
            non_attentive=self.non_attentive,
            location_specific_attention=self.location_specific_attention,
            sampling_rate=self.sampling_rate,
            hop_length=self.hop_length,
        )  # keep higher than 5 to make clips not stretch on
        print("campaign with this game vp3")
        model = Tacotron2(self.hparams)
        print("campaign with this game vp7")
        if self.device == "cuda":
            model = model.cuda()
        print("campaign with this game v1")
        if self.distributed_run:
            #NOTE (Sam) find_unused_parameters reduces unreduced losses that we might have missed
            model = DDP(model, device_ids=[self.rank], find_unused_parameters=True)
        print("campaign with this game v2")
        optimizer = torch.optim.Adam(
            model.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay,
        )
        start_epoch = 0

        if self.warm_start_name:
            if self.distributed_run:
                module, optimizer, start_epoch = self.warm_start(
                    model.module, optimizer
                )
                model.module = module
            else:
                model, optimizer, start_epoch = self.warm_start(model, optimizer)

        if self.fp16_run:
            scaler = GradScaler()
        print("campaign with this game")
        start_time, previous_start_time = time.perf_counter(), time.perf_counter()
        for epoch in range(start_epoch, self.epochs):
            #             train_loader, sampler, collate_fn = self.adjust_frames_per_step(
            #                 model, train_loader, sampler, collate_fn
            #             )
            if self.distributed_run:
                sampler.set_epoch(epoch)
            for batch_idx, batch in enumerate(train_loader):
                previous_start_time = start_time
                start_time = time.perf_counter()
                self.global_step += 1
                model.zero_grad()
                if self.distributed_run:
                    X, y = model.module.parse_batch(batch)
                else:
                    X, y = model.parse_batch(batch)
                if self.fp16_run:
                    with autocast():
                        y_pred = model(X)
                        if self.location_specific_attention:
                            (
                                loss,
                                loss_batch,
                                mel_loss,
                                mel_loss_batch,
                                gate_loss,
                                gate_loss_batch,
                            ) = criterion(y_pred, y)
                        if self.non_attentive:
                            (
                                loss,
                                loss_batch,
                                mel_loss,
                                mel_loss_batch,
                                duration_loss,
                                duration_loss_batch,
                            ) = criterion(y_pred, y)
                else:
                    y_pred = model(X)
                    if self.location_specific_attention:
                        (
                            loss,
                            loss_batch,
                            mel_loss,
                            mel_loss_batch,
                            gate_loss,
                            gate_loss_batch,
                        ) = criterion(y_pred, y)
                    if self.non_attentive:
                        (
                            loss,
                            loss_batch,
                            mel_loss,
                            mel_loss_batch,
                            duration_loss,
                            duration_loss_batch,
                        ) = criterion(y_pred, y)
                if self.distributed_run:
                    reduced_mel_loss = reduce_tensor(mel_loss, self.world_size).item()
                    reduced_mel_loss_batch = reduce_tensor(
                        mel_loss_batch, self.world_size
                    )
                    if self.location_specific_attention:
                        reduced_gate_loss = reduce_tensor(
                            gate_loss, self.world_size
                        ).item()
                        reduced_gate_loss_batch = reduce_tensor(
                            gate_loss_batch, self.world_size
                        )
                        reduced_loss = reduced_mel_loss + reduced_gate_loss
                    if self.non_attentive:
                        reduced_gate_loss = None
                        reduced_gate_loss_batch = None
                        reduced_duration_loss = reduce_tensor(
                            duration_loss, self.world_size
                        ).item()
                        reduced_duration_loss_batch = reduce_tensor(
                            duration_loss_batch, self.world_size
                        ).item()
                        reduced_loss = reduced_mel_loss + reduced_duration_loss
                else:
                    reduced_mel_loss = mel_loss  # .item()
                    reduced_mel_loss_batch = mel_loss_batch.detach()
                    if self.location_specific_attention:
                        reduced_gate_loss = gate_loss.item()
                        reduced_gate_loss_batch = gate_loss_batch.detach()
                        reduced_loss = reduced_mel_loss + reduced_gate_loss
                    if self.non_attentive:
                        reduced_gate_loss = None
                        reduced_gate_loss_batch = None
                        reduced_duration_loss = duration_loss.item()
                        reduced_duration_loss_batch = duration_loss_batch.detach()
                        reduced_loss = reduced_mel_loss + reduced_duration_loss

                if self.fp16_run:
                    scaler.scale(loss).backward()
                    scaler.unscale_(optimizer)
                    grad_norm = torch.nn.utils.clip_grad_norm(
                        model.parameters(), self.grad_clip_thresh
                    )
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    loss.backward()
                    grad_norm = torch.nn.utils.clip_grad_norm(
                        model.parameters(), self.grad_clip_thresh
                    )
                    optimizer.step()
                step_duration_seconds = time.perf_counter() - start_time
                log_start = time.time()
                self.log_training(
                    model,
                    X,
                    y_pred,
                    y,
                    reduced_loss,
                    reduced_mel_loss,
                    reduced_gate_loss,
                    reduced_mel_loss_batch,
                    reduced_gate_loss_batch,
                    grad_norm,
                    step_duration_seconds,
                )
                log_stop = time.time()
                if self.location_specific_attention:
                    log_str = f"epoch: {epoch}/{self.epochs} | batch: {batch_idx}/{len(train_loader)} | loss: {reduced_loss:.2f} | mel: {reduced_mel_loss:.2f} | gate: {reduced_gate_loss:.3f} | t: {start_time - previous_start_time:.2f}s | w: {(time.perf_counter() - train_start_time)/(60*60):.2f}h"
                if self.non_attentive:
                    log_str = f"epoch: {epoch}/{self.epochs} | batch: {batch_idx}/{len(train_loader)} | loss: {reduced_loss:.2f} | mel: {reduced_mel_loss:.2f} | t: {start_time - previous_start_time:.2f}s | w: {(time.perf_counter() - train_start_time)/(60*60):.2f}h"
                if self.distributed_run:
                    log_str += f" | rank: {self.rank}"
                print(log_str)
            if epoch % self.epochs_per_checkpoint == 0:
                self.save_checkpoint(
                    f"tacotron2_{epoch}",
                    model=model,
                    optimizer=optimizer,
                    iteration=epoch,
                    learning_rate=self.learning_rate,
                    global_step=self.global_step,
                )

            # There's no need to validate in debug mode since we're not really training.
            if self.debug:
                continue
            self.validate(
                model=model,
                val_set=val_set,
                collate_fn=collate_fn,
                criterion=criterion,
            )

    def validate(self, **kwargs):
        val_start_time = time.perf_counter()

        model = kwargs["model"]
        val_set = kwargs["val_set"]
        collate_fn = kwargs["collate_fn"]
        criterion = kwargs["criterion"]
        sampler = DistributedSampler(val_set) if self.distributed_run else None
        (total_loss, total_mel_loss, total_gate_loss, total_duration_loss) = (
            0,
            0,
            0,
            0,
        )
        total_steps = 0
        model.eval()
        speakers_val = []
        batch_mel_loss = []
        batch_gate_loss = []
        batch_duration_loss = []
        with torch.no_grad():
            val_loader = DataLoader(
                val_set,
                sampler=sampler,
                shuffle=False,
                batch_size=self.batch_size,
                collate_fn=collate_fn,
            )
            for batch in val_loader:
                total_steps += 1
                if self.distributed_run:
                    X, y = model.module.parse_batch(batch)
                    speakers_val.append(X.speaker_ids)
                else:
                    X, y = model.parse_batch(batch)
                    speakers_val.append(X.speaker_ids)
                y_pred = model(X)
                if self.location_specific_attention:
                    (
                        loss,
                        loss_batch,
                        mel_loss,
                        mel_loss_batch,
                        gate_loss,
                        gate_loss_batch,
                    ) = criterion(y_pred, y)
                if self.non_attentive:
                    (
                        loss,
                        loss_batch,
                        mel_loss,
                        mel_loss_batch,
                        duration_loss,
                        duration_loss_batch,
                    ) = criterion(y_pred, y)
                if self.distributed_run:
                    reduced_mel_loss = reduce_tensor(mel_loss, self.world_size).item()
                    reduced_mel_loss_batch = reduce_tensor(
                        mel_loss_batch, self.world_size
                    )
                    if self.location_specific_attention:
                        reduced_gate_loss = reduce_tensor(
                            gate_loss, self.world_size
                        ).item()
                        reduced_gate_loss_batch = reduce_tensor(
                            gate_loss_batch, self.world_size
                        ).detach()
                        reduced_loss = reduced_mel_loss + reduced_gate_loss

                    if self.non_attentive:
                        reduced_duration_loss = reduce_tensor(
                            duration_loss, self.world_size
                        ).item()
                        reduced_loss = reduced_mel_loss + reduced_duration_loss

                else:
                    reduced_mel_loss = mel_loss  # .item()
                    reduced_mel_loss_batch = mel_loss_batch.detach()
                    if self.location_specific_attention:
                        reduced_gate_loss = gate_loss.item()
                        reduced_gate_loss_batch = gate_loss_batch.detach()
                    if self.non_attentive:
                        reduced_duration_loss = duration_loss.item()
                        reduced_duration_loss_batch = duration_loss_batch.detach()

                batch_mel_loss.append(reduced_mel_loss_batch)
                total_mel_loss += reduced_mel_loss
                if self.location_specific_attention:
                    batch_gate_loss.append(reduced_gate_loss_batch)
                    reduced_val_loss = reduced_mel_loss + reduced_gate_loss
                    total_gate_loss += reduced_gate_loss

                if self.non_attentive:
                    batch_duration_loss.append(reduced_duration_loss)
                    reduced_val_loss = reduced_mel_loss + reduced_duration_loss
                    total_duration_loss += reduced_duration_loss

                total_loss += reduced_val_loss

            mean_mel_loss = total_mel_loss / total_steps
            if self.location_specific_attention:
                mean_gate_loss = total_gate_loss / total_steps
                batch_gate_loss = torch.hstack(batch_gate_loss)
            else:
                # NOTE (Sam): for convenience... really should have a namedtuple loss class
                mean_gate_loss = torch.tensor([0]).float()
                batch_gate_loss = torch.zeros((self.batch_size, 1)).float()
            mean_loss = total_loss / total_steps
            batch_mel_loss = torch.hstack(batch_mel_loss)

            speakers_val = torch.hstack(speakers_val)
            self.log_validation(
                X,
                y_pred,
                y,
                mean_loss,
                mean_mel_loss,
                mean_gate_loss,
                batch_gate_loss,
                batch_mel_loss,
                speakers_val,
            )

        model.train()

        if self.location_specific_attention:
            val_log_str = f"Validation loss: {mean_loss:.2f} | mel: {mel_loss:.2f} | gate: {mean_gate_loss:.3f} | t: {time.perf_counter() - val_start_time:.2f}s"
        if self.non_attentive:
            val_log_str = f"Validation loss: {mean_loss:.2f} | mel: {mel_loss:.2f} |  t: {time.perf_counter() - val_start_time:.2f}s"

    @property
    def val_dataset_args(self):

        args = dict(**self.training_dataset_args)
        args["audiopaths_and_text"] = self.val_audiopaths_and_text
        return args

    @property
    def training_dataset_args(self):
        return {
            "audiopaths_and_text": self.training_audiopaths_and_text,
            "text_cleaners": self.text_cleaners,
            "p_arpabet": self.p_arpabet,
            "n_mel_channels": self.n_mel_channels,
            "sampling_rate": self.sampling_rate,
            "mel_fmin": self.mel_fmin,
            "mel_fmax": self.mel_fmax,
            "filter_length": self.filter_length,
            "hop_length": self.hop_length,
            "win_length": self.win_length,
            "symbol_set": self.symbol_set,
            "max_wav_value": self.max_wav_value,
            "pos_weight": self.pos_weight,
            "compute_gst": self.compute_gst,
            "include_durations": self.include_durations,
            "compute_durations": self.compute_durations,
        }

In [3]:
# export
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.trainer.base import DEFAULTS as TRAINER_DEFAULTS
from uberduck_ml_dev.models.tacotron2.tacotron2 import DEFAULTS as TACOTRON2_DEFAULTS

config = TRAINER_DEFAULTS.values()
config.update(TACOTRON2_DEFAULTS.values())
DEFAULTS = HParams(**config)

In [4]:
# rank = None
# device_count = None
# trainer = Tacotron2Trainer(DEFAULTS, rank=rank, world_size=device_count)
# trainer.train()

In [5]:
# # skip
# import json
# import torch.distributed as dist

# config = DEFAULTS.values()
# config["batch_size"] = 4
# with open("test/fixtures/ljtest/taco2_lj2lj.json") as f:
#     config.update(json.load(f))
# config["distributed_run"] = True
# hparams = HParams(**config)
# rank = 0
# device_count = 4
# trainer = Tacotron2Trainer(hparams, rank=rank, world_size=device_count)
# # debug DDP launching
# self = trainer
# trainer.train()

TTSTrainer start 73523.630250416
Initializing trainer with hparams:
{'attention_dim': 128,
 'attention_location_kernel_size': 31,
 'attention_location_n_filters': 32,
 'attention_rnn_dim': 1024,
 'batch_size': 16,
 'checkpoint_name': None,
 'checkpoint_path': 'test/fixtures/results/checkpoints',
 'coarse_n_frames_per_step': None,
 'compute_durations': False,
 'cudnn_enabled': True,
 'dataset_path': './dataset',
 'debug': False,
 'decay_rate': 8000,
 'decay_start': 15000,
 'decoder_rnn_dim': 1024,
 'distributed_run': True,
 'encoder_embedding_dim': 512,
 'encoder_kernel_size': 5,
 'encoder_n_convolutions': 3,
 'epochs': 5,
 'epochs_per_checkpoint': 4,
 'filter_length': 1024,
 'fp16_run': False,
 'gate_threshold': 0.5,
 'grad_clip_thresh': 1.0,
 'gst_type': None,
 'has_speaker_embedding': False,
 'hop_length': 256,
 'ignore_layers': ['speaker_embedding.weight'],
 'include_durations': False,
 'include_f0': False,
 'learning_rate': 0.001,
 'location_specific_attention': True,
 'log_dir': '

In [ ]:
# skip

import torch.distributed as dist

dist.init_process_group(
    "nccl", init_method="tcp://localhost:54321", rank=1, world_size=4,
)

In [ ]:
# skip

import json

config = DEFAULTS.values()
config["batch_size"] = 4
with open("test/fixtures/ljtest/taco2_lj2lj.json") as f:
    config.update(json.load(f))
config["distributed_run"] = False
hparams = HParams(**config)
rank = 0
device_count = 4
trainer = Tacotron2Trainer(hparams, rank=rank, world_size=device_count)
trainer.train()

In [ ]:
# def initialize_loader(
#     self,
#     include_f0: bool = False,
#     include_durations: bool = False,
#     n_frames_per_step: int = 1,
# ):
#     train_set = TextMelDataset(
#         **self.training_dataset_args,
#         debug=self.debug,
#         debug_dataset_size=self.batch_size,
#     )
#     val_set = TextMelDataset(
#         **self.val_dataset_args, debug=self.debug, debug_dataset_size=self.batch_size,
#     )
#     collate_fn = TextMelCollate(
#         n_frames_per_step=n_frames_per_step,
#         include_f0=include_f0,
#         include_durations=include_durations,
#     )
#     sampler = None
#     if self.distributed_run:
#         self.init_distributed()
#         sampler = DistributedSampler(train_set, rank=self.rank)
#     train_loader = DataLoader(
#         train_set,
#         batch_size=self.batch_size,
#         shuffle=(sampler is None),
#         sampler=sampler,
#         collate_fn=collate_fn,
#     )
#     return train_set, val_set, train_loader, sampler, collate_fn